In [1]:
import os
import json
import numpy as np

In [2]:
def get_data(txt_file):
    with open(txt_file, "r") as f:
        gt = json.load(f)
    
    data={example['id'] : example for example in gt}

    return data


In [3]:
train_set=get_data('data/training/training_set_task3.txt')
val_set=get_data('data/dev/dev_set_task3_labeled.txt')
test_set=get_data('data/testing/test_set_task3.txt')


In [9]:
def generate_prompt(fewshot_ids,split):
    prompt='''The following is a conversation between a curious human and AI assistant. The assistant identify persuasion techniques in image and text given by the human. 
    This is a list of persuasion techniques: ['Appeal to authority', 'Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 
    'Glittering generalities (Virtue)', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 
    'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché', 'Whataboutism', 'Bandwagon', 'Transfer', 'Appeal to (Strong) Emotions']\n'''

    fewshot_image_path=[]

    DATA_PATH='data/'+split+'/'

    for id in fewshot_ids:
        text=train_set[id]['text']
        labels=str(train_set[id]['labels'])
        fewshot_image_path.append(train_set[id]['image'])

        prompt+=\
            f'''
            Human: <image>
            Human: \"{text}\" Please select the techniques used in the image and the text.
            AI: {labels}'''
    
    return prompt,fewshot_image_path


In [10]:
few_show_examples=['189', '186_batch_2', '136', '85', '45_batch_2']

print(generate_prompt(few_show_examples,'train')[0])

The following is a conversation between a curious human and AI assistant. The assistant identify persuasion techniques in image and text given by the human. 
    This is a list of persuasion techniques: ['Appeal to authority', 'Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 
    'Glittering generalities (Virtue)', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 
    'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché', 'Whataboutism', 'Bandwagon', 'Transfer', 'Appeal to (Strong) Emotions']

            Human: <image>
            Human: "This is not an accident!" Please select the techniques used in the image and the text.
            AI: ['Reductio ad hitlerum', 'Smears', 'Transfer']
            Human: <image>

In [11]:
def generate_data(data, few_show_examples,split):

    prompt, fewshot_image_path=generate_prompt(few_show_examples, split)

    data_examples=[]

    DATA_PATH='data/'+split+'/'
    
    for id in data:
        image_path=DATA_PATH+data[id]['image']
        text=data[id]['text']
        labels=data[id]['labels']

        example_text=\
            f'''
            Human: <image>
            Human: \"{text}\" Please select the techniques used in the image and the text.
            AI: {labels}'''
        
        example={}
        example['task_type'] = 'llava_sft'
        example['image'] = fewshot_image_path + [image_path]
        example['text'] = prompt + example_text
        
        data_examples.append(example)

    return data_examples


In [12]:
# few_show_examples=['189', '186_batch_2', '136', '85', '45_batch_2']
few_show_examples=[]

train_json=generate_data(train_set,few_show_examples, 'train')
train_file_path = "trainset.jsonl"
val_json=generate_data(val_set,few_show_examples, 'dev')
val_file_path = "valset.jsonl"

with open(train_file_path, 'w') as json_file:
    for entry in train_json:
        json.dump(entry,json_file)
        json_file.write('\n')
    # json.dump(train_json, json_file, indent=4) 
with open(val_file_path, 'w') as json_file:
    for entry in val_json:
        json.dump(entry,json_file)
        json_file.write('\n')
    # json.dump(val_json, json_file, indent=4) 

In [13]:
for keys in train_json[0]:
    print(type(train_json[0][keys]))
    # print(repr(train_json[0][keys]))
    print(train_json[0][keys])

<class 'str'>
llava_sft
<class 'list'>
['data/train/128_image.png']
<class 'str'>
The following is a conversation between a curious human and AI assistant. The assistant identify persuasion techniques in image and text given by the human. 
    This is a list of persuasion techniques: ['Appeal to authority', 'Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 
    'Glittering generalities (Virtue)', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 
    'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché', 'Whataboutism', 'Bandwagon', 'Transfer', 'Appeal to (Strong) Emotions']

            Human: <image>
            Human: "THERE ARE ONLY TWO GENDERS

FEMALE 

MALE
" Please select the techniques used in the image and